In [27]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import random
import itertools
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim

np.random.seed(0)

In [28]:
# definição dos grupos de filiais pra facilitar a vida

df = pd.read_csv("../../dados/distancias_todas_combinacoes.csv")

CD = 'CENTRO DE DISTRIBUICAO'

todas_com_cd = df

todas_sem_cd = df[(df['Filial_A'] != CD) 
                      & (df['Filial_B'] != CD)]

rj_com_cd = df[(df['Estado_A'] == 'RJ')|(df['Estado_B'] == 'RJ')]  # uniao de E_A e E_B == RJ

rj_sem_cd = todas_sem_cd[(todas_sem_cd['Estado_A'] == 'RJ')|(todas_sem_cd['Estado_B'] == 'RJ')]  # uniao de E_A e E_B == RJ \{filial = CD}

sp_sem_cd = todas_sem_cd[(todas_sem_cd['Estado_A'] == 'SP')|(todas_sem_cd['Estado_B'] == 'SP')] # uniao de E_A e E_B == SP

cd_sp = df[((df['Estado_A'] == 'SP') & (df['Filial_B'] == CD))
           | ((df['Estado_B'] == 'SP') & (df['Filial_A'] == CD))]
sp_com_cd = pd.concat((cd_sp, sp_sem_cd), ignore_index=True)  # uniao de E_A e E_B == SP  +  {A = CD e E_B = SP}

lojas_A = todas_sem_cd[['Estado_A', 'Filial_A']].rename(columns={'Estado_A': 'Estado', 'Filial_A': 'Filial'})
lojas_B = todas_sem_cd[['Estado_B', 'Filial_B']].rename(columns={'Estado_B': 'Estado', 'Filial_B': 'Filial'})

um_de_cada = pd.concat([lojas_A, lojas_B]) \
                     .drop_duplicates() \
                     .sort_values(by=['Estado', 'Filial']) \
                     .drop_duplicates(subset=['Estado']) \
                     .reset_index(drop=True)

estado_sem_cd = todas_sem_cd[(todas_sem_cd['Filial_A'].isin(um_de_cada['Filial'].tolist())) &
                             (todas_sem_cd['Filial_B'].isin(um_de_cada['Filial'].tolist()))]

cd_filiais = df[(df['Filial_A'] == CD) & (df['Filial_B'].isin(um_de_cada['Filial'].tolist()))]

estado_com_cd = pd.concat([estado_sem_cd, cd_filiais], ignore_index=True)

___

#### Temos uma função $F_A(B)$ = valor de B no ranking de A:

##### 1) Caso A→B (A como provedor):

$$F_A(B) = (\frac{1}{Leadtime} + \frac{1}{CustoFrete}) * (ContRup*p + ContFalta*q)$$

##### 2) Caso CD→B (CD como provedor): 

$$F_{CD}(B) = α * (\frac{1}{Leadtime} + \frac{1}{CustoFrete}) * (ContRup*p + ContFalta*q)$$

onde $α$ é um fator preferencia do CD ser o provedor.

___

#### Utilize uma das variáveis para a simulação:

* `todas_sem_cd` para utilizar todas as filiais do Brasil.
* `todas_com_cd` para utilizar todas as filiais do Brasil e o Centro de Distribuição.
* `rj_sem_cd` para utilizar todas as filiais do Rio de Janeiro.
* `rj_com_cd` para utilizar todas as filiais do Rio de Janeiro e o Centro de Distribuição.
* `sp_sem_cd` para utilizar todas as filiais de São Paulo.
* `sp_com_cd` para utilizar todas as filiais de São Paulo e o Centro de Distribuição.
* `estado_sem_cd` para utilizar uma filial e cada estado.
* `estado_com_cd` para utilizar uma filial e cada estado e o Centro de Distribuição.

#### Não esqueça de especificar se o CD está incluso ou não (True/False)

In [29]:
# Defina todas as variáveis abaixo!

filiais = estado_com_cd # qual conjunto de variáveis usar 
cd = False # booleano se considera ou não o CD no grafo # acho que nem vai ser necessário

c = 1 # constante de tempo de transporte por km 
k_filial = 0.5  # constante de custo do frete em reais por km saindo de A
k_cd = 0.2  # constante de custo do frete em reais por km saindo do CD
p = 2  # peso se a filial está em ruptura
q = 1  # peso se a filial está em falta
alpha = 2  # fator de preferencia do CD ser o provedor

params = {'c': c, 'k_filial': k_filial, 'k_cd': k_cd, 'p': p, 'q': q, 'alpha': alpha}


In [30]:
def classificar_filiais(df):
    """
    Classifica um DataFrame em DataFrames de provedores e receptoras.

    Argumentos:
    df -- DataFrame com as colunas: 
                 ['Filial', 'Tipo', 'Est_disp', 'Alvo']
    
    Retorna:
    (df_provedores, df_receptoras) -- Uma tupla contendo os dois DataFrames.
    """
   
    # Filais provedoras
    df_prov = df.copy()
    df_prov['Vol_Exc'] = np.where((df_prov['Tipo'] == 'CD'), # se é o CD
                                   df_prov['Est_disp'],      # o valor do excesso = estoque disponivel
                                   (df_prov['Est_disp'] - df_prov['Alvo']).clip(lower=0))   # se não, a diferença entre estoque e alvo 
    df_provedores = df_prov[df_prov['Vol_Exc'] > 0].copy()
    df_provedores = df_provedores[['Filial', 'Tipo', 'Vol_Exc']]

    # Filiais receptoras
    df_rec = df[df['Tipo'] == 'Filial'].copy()
    df_receptoras = df_rec[df_rec['Est_disp'] < df_rec['Alvo']].copy() # pega só as com estoque abaixo do alvo
    df_receptoras['Vol_Falta'] = (df_receptoras['Alvo'] - df_receptoras['Est_disp']) # Vol_Falta é o quanto falta para chegar no alvo
    df_receptoras['Cont_Rup'] = np.where(df_receptoras['Est_disp'] == 0, 1, 0) # 
    df_receptoras['Cont_Falta'] = np.where(df_receptoras['Est_disp'] > 0, 1, 0)

    colunas_finais_rec = ['Filial', 'Vol_Falta', 'Cont_Rup', 'Cont_Falta']
    df_receptoras = df_receptoras[colunas_finais_rec]
    
    return df_provedores, df_receptoras

In [31]:
def get_distancia(a, b, filiais):
    """Função para buscar distâncias no df"""
    if a == b:
        return 0
    dist_df = filiais[
        ((filiais['Filial_A'] == a) & (filiais['Filial_B'] == b)) |
        ((filiais['Filial_A'] == b) & (filiais['Filial_B'] == a))
    ]
    if dist_df.empty:
        return np.inf  # Retorna infinito se a rota não existe
    return dist_df['Distancia_km'].values[0]

In [32]:
def calcular_ranking_F(
    provedor, 
    receptor, 
    status_receptor, 
    filiais, 
    params
):
    """Calcula o score F_A(B) ou F_CD(B) para um par (A, B)."""
    
    dist = get_distancia(provedor, receptor, filiais)
    if dist == np.inf or dist == 0:
        return 0  # Score 0 se a rota não existe ou é para si mesmo

    leadtime = params['c'] * dist  # calculo do leadtime
    
    if provedor == CD:
        custo_frete = params['k_cd'] * dist # calculo do custo (CD)
    else:
        custo_frete = params['k_filial'] * dist # calculo do custo (filial)

    if status_receptor == "Ruptura":
        cont_rup = 1
        cont_falta = 0
    elif status_receptor == "Falta":
        cont_rup = 0
        cont_falta = 1
    else:
        return 0 
    # finalmente o calculo da formula 
    f_AB = ((1 / (leadtime+1)) + (1 / (custo_frete+1))) * ((cont_rup * params['p']) + (cont_falta * params['q']))
    
    if provedor == CD: # multiplica por alpha se o CD for o provedor 
        return params['alpha'] * f_AB
    else:
        return f_AB

In [33]:
def montar_matriz_T(lista_provedores, df_receptoras, filiais, params):
    """
    Monta a Matriz T (DataFrame) onde T[i, j] = F_Ai(B_j).
    
    - lista_provedores: Uma lista de nomes de filiais provedoras (ex: ['CD', 'A1', 'A2'])
    - df_receptoras: O DataFrame de filiais B (receptoras)
    """
    
    matriz_data = []
    
    # Itera sobre cada receptoras (colunas)
    for _, receptor in df_receptoras.iterrows():
        filial_b = receptor['Filial']
        status_b = receptor['Status']
        
        scores_para_b = {'Receptora': filial_b}
        
        # Itera sobre cada PROVEDOR (linhas)
        for filial_a in lista_provedores:
            
            # Calcula o score do par (A, B)
            score = calcular_ranking_F(
                provedor=filial_a,
                receptor=filial_b,
                status_receptor=status_b,
                filiais=filiais,
                params=params
            )
            scores_para_b[filial_a] = score
            
        matriz_data.append(scores_para_b)
        
    # Converte os dados em um DataFrame e o "pivota" (transpõe)
    # Queremos Provedores (A_i) nas linhas e Receptoras (B_j) nas colunas
    
    matriz_T = pd.DataFrame(matriz_data)
    matriz_T = matriz_T.set_index('Receptora').T
    
    return matriz_T

In [44]:
import sys
import json
from pathlib import Path
PROJECT_ROOT = Path.cwd().resolve().parent
sys.path.append(str(PROJECT_ROOT))

from gcpUtils import auth 
from gcpUtils import bigQuery as bQ


arquivoFiliais = "../../dados/filiais_inferior_30.json"
cred = auth.getCredentials("../../bd/planejamento-animale-292719-296d49ccdea6.json")

In [ ]:
tabela_bigquery = 'planejamento-animale-292719.checklists_rollout.ANIMALE_checklist'
data0 = '2025-08-16'

with open(arquivoFiliais, 'r', encoding='utf-8') as f:
    dados_filiais = json.load(f)
    
df_filiais = pd.DataFrame(dados_filiais)

lista_nomes_filiais = df_filiais['FILIAL'].unique().tolist()

filiaisQuery = f"('{ "','".join(lista_nomes_filiais)}')"

query = f"""
    SELECT SKU, FILIAL, VELOCIDADE_VENDA, ALVO, TRANSITO, EST_DISP, EST_TOTAL, 
    CONT_RUPTURA, CONT_FALTA, CONT_EXCESSO, VOLUME_EXCESSO, CATEGORIA_EST_TOTAL, CATEGORIA_EST_DISP
    FROM {tabela_bigquery}
    WHERE FILIAL IN {filiaisQuery}
    AND DATA = '{data0}'
"""
df_resultado = bQ.tableToPandas(query, 'planejamento-animale-292719', cred)
print(df_resultado.head())



/home/samyra/fgv/Project_AZZAS2154/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


                      SKU                 FILIAL  VELOCIDADE_VENDA  ALVO  \
0   27.03.1447-1511-TAM_1  ANIMALE ALPHAVILLE CM               0.0   1.0   
1  07.20.7684-03032-TAM_1     ANIMALE ARACAJU CM               0.0   2.0   
2   11.05.0565-1802-TAM_1     ANIMALE ARACAJU CM               0.0   2.0   
3   11.05.0572-1419-TAM_1     ANIMALE ARACAJU CM               0.0   2.0   
4   27.03.1447-1511-TAM_1     ANIMALE ARACAJU CM               0.0   1.0   

   PRESENTE  TRANSITO  EST_TOTAL  CONT_RUPTURA  CONT_FALTA  CONT_EXCESSO  \
0       5.0       0.0        5.0             0           0             1   
1       8.0       8.0        8.0             0           0             0   
2       6.0       6.0        6.0             0           0             0   
3       6.0       6.0        6.0             0           0             0   
4       6.0       0.0        6.0             0           0             1   

   VOLUME_EXCESSO  REGULADOR CATEGORIA_EST_TOTAL   CATEGORIA_EST_DISP  
0             

In [51]:
#inicializar bd
df_resultado.to_csv('../../dados/simulacao_diaria.csv', index=False)




In [ ]:
def simulacao_inicial():
    pass